### Laboratorio 6 - Deep learning

- Alejandra Guzmán 20562
- Jorge Caballeros 20009



In [2]:
import pandas as pd

users_df = pd.read_csv('Users.csv')
books_df = pd.read_csv('Books.csv')
ratings_df = pd.read_csv('Ratings.csv')

users_head = users_df.head()
books_head = books_df.head()
ratings_head = ratings_df.head()

(users_head, books_head, ratings_head)


C:\Users\alegu\AppData\Local\Temp\ipykernel_5500\3780130330.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv('Books.csv')


(   User-ID                            Location   Age
 0        1                  nyc, new york, usa   NaN
 1        2           stockton, california, usa  18.0
 2        3     moscow, yukon territory, russia   NaN
 3        4           porto, v.n.gaia, portugal  17.0
 4        5  farnborough, hants, united kingdom   NaN,
          ISBN                                         Book-Title  \
 0  0195153448                                Classical Mythology   
 1  0002005018                                       Clara Callan   
 2  0060973129                               Decision in Normandy   
 3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
 4  0393045218                             The Mummies of Urumchi   
 
             Book-Author Year-Of-Publication                   Publisher  \
 0    Mark P. O. Morford                2002     Oxford University Press   
 1  Richard Bruce Wright                2001       HarperFlamingo Canada   
 2          Carlo D'Este       

In [3]:
# EDA - Users Dataset
users_info = {
    "Users Summary": users_df.describe(include='all'),
    "Users Missing Values": users_df.isnull().sum(),
    "Unique Locations": users_df['Location'].nunique(),
    "Age Distribution": users_df['Age'].describe()
}

# EDA - Books Dataset
books_info = {
    "Books Summary": books_df.describe(include='all'),
    "Books Missing Values": books_df.isnull().sum(),
    "Unique Authors": books_df['Book-Author'].nunique(),
    "Publication Year Distribution": books_df['Year-Of-Publication'].describe()
}

# EDA - Ratings Dataset
ratings_info = {
    "Ratings Summary": ratings_df.describe(),
    "Ratings Missing Values": ratings_df.isnull().sum(),
    "Rating Distribution": ratings_df['Book-Rating'].value_counts()
}

(users_info, books_info, ratings_info)


({'Users Summary':              User-ID                         Location            Age
  count   278858.00000                           278858  168096.000000
  unique           NaN                            57339            NaN
  top              NaN  london, england, united kingdom            NaN
  freq             NaN                             2506            NaN
  mean    139429.50000                              NaN      34.751434
  std      80499.51502                              NaN      14.428097
  min          1.00000                              NaN       0.000000
  25%      69715.25000                              NaN      24.000000
  50%     139429.50000                              NaN      32.000000
  75%     209143.75000                              NaN      44.000000
  max     278858.00000                              NaN     244.000000,
  'Users Missing Values': User-ID          0
  Location         0
  Age         110762
  dtype: int64,
  'Unique Locations': 57339

### PreProcesamiento de la data

In [4]:
# Users Dataset Preprocessing
# Remove unrealistic ages
users_df_cleaned = users_df[(users_df.Age >= 5) & (users_df.Age <= 100)]

# Fill missing ages with median value of the cleaned dataset
median_age = users_df_cleaned.Age.median()
users_df_cleaned.Age.fillna(median_age, inplace=True)

# Books Dataset Preprocessing
# Convert 'Year-Of-Publication' to numeric, coerce errors to NaN, and then convert NaNs to the median year
books_df['Year-Of-Publication'] = pd.to_numeric(books_df['Year-Of-Publication'], errors='coerce')
median_year = books_df['Year-Of-Publication'].median()
books_df['Year-Of-Publication'].fillna(median_year, inplace=True)
books_df['Year-Of-Publication'] = books_df['Year-Of-Publication'].astype(int)

# Replace missing values in 'Book-Author' and 'Publisher' with 'Unknown'
books_df['Book-Author'].fillna('Unknown', inplace=True)
books_df['Publisher'].fillna('Unknown', inplace=True)

# Ratings Dataset Preprocessing
# For now, let's keep the ratings as they are, including the 0 ratings, and decide on the approach when building the recommendation system

# Checking the results of the preprocessing
preprocessed_data = {
    "Users Cleaned": users_df_cleaned.head(),
    "Users Missing Ages After Cleaning": users_df_cleaned['Age'].isnull().sum(),
    "Books Cleaned": books_df.head(),
    "Books Missing Values After Cleaning": books_df.isnull().sum()
}

preprocessed_data


C:\Users\alegu\AppData\Local\Temp\ipykernel_5500\2994511849.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_df_cleaned.Age.fillna(median_age, inplace=True)


{'Users Cleaned':     User-ID                        Location   Age
 1         2       stockton, california, usa  18.0
 3         4       porto, v.n.gaia, portugal  17.0
 5         6   santa monica, california, usa  61.0
 9        10      albacete, wisconsin, spain  26.0
 10       11  melbourne, victoria, australia  14.0,
 'Users Missing Ages After Cleaning': 0,
 'Books Cleaned':          ISBN                                         Book-Title  \
 0  0195153448                                Classical Mythology   
 1  0002005018                                       Clara Callan   
 2  0060973129                               Decision in Normandy   
 3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
 4  0393045218                             The Mummies of Urumchi   
 
             Book-Author  Year-Of-Publication                   Publisher  \
 0    Mark P. O. Morford                 2002     Oxford University Press   
 1  Richard Bruce Wright                 2001   

### Sistema de recomendacion basado en contenido

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Load a smaller subset of the Books dataset for content-based recommendation system
# Due to memory constraints, we will work with the first 1000 books
books_df_subset = pd.read_csv('Books.csv', nrows=1000)

# Use TF-IDF to convert the 'Book-Title' into a matrix of TF-IDF features
tfidf_vectorizer_subset = TfidfVectorizer(stop_words='english')
tfidf_matrix_subset = tfidf_vectorizer_subset.fit_transform(books_df_subset['Book-Title'].fillna(''))

# Compute the cosine similarity matrix based on the tfidf_matrix
cosine_sim_matrix_subset = linear_kernel(tfidf_matrix_subset, tfidf_matrix_subset)

# Function to get recommendations based on the cosine similarity score of book titles
def get_content_based_recommendations_subset(title, cosine_sim=cosine_sim_matrix_subset):
    # Get the index of the book that matches the title
    idx = books_df_subset.index[books_df_subset['Book-Title'] == title].tolist()[0]

    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar books
    sim_scores = sim_scores[1:11]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar books
    return books_df_subset['Book-Title'].iloc[book_indices]

# Testing the content-based recommendation function with a book title from the subset dataset
test_book_title_subset = books_df_subset['Book-Title'].iloc[0]  # Get the first book title for testing
content_based_recommendations_subset = get_content_based_recommendations_subset(test_book_title_subset)

content_based_recommendations_subset


1                                          Clara Callan
2                                  Decision in Normandy
3     Flu: The Story of the Great Influenza Pandemic...
4                                The Mummies of Urumchi
5                                The Kitchen God's Wife
6     What If?: The World's Foremost Military Histor...
7                                       PLEADING GUILTY
8     Under the Black Flag: The Romance and the Real...
9               Where You'll Find Me: And Other Stories
10                          Nights Below Station Street
Name: Book-Title, dtype: object

In [6]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# Load a smaller subset of the Ratings dataset for collaborative filtering recommendation system
# We will work with the first 1000 ratings to fit within memory constraints
ratings_df_subset = pd.read_csv('Ratings.csv', nrows=1000)

# Create the user-item interaction matrix
interaction_matrix = ratings_df_subset.pivot(index='User-ID', columns='ISBN', values='Book-Rating').fillna(0)

# Convert the user-item interaction matrix to a sparse matrix format to save memory
sparse_interaction_matrix = csr_matrix(interaction_matrix.values)

# Initialize and fit the NearestNeighbors model (which is a memory-based collaborative filtering model)
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
model_knn.fit(sparse_interaction_matrix)

# Function to get book recommendations for a user using the collaborative filtering model
def get_collaborative_filtering_recommendations(user_id, interaction_matrix, model_knn):
    # Get the index of the user for which the recommendations are to be made
    user_idx = interaction_matrix.index.tolist().index(user_id)

    # Get the user interaction vector for the user
    user_interaction_vector = interaction_matrix.values[user_idx, :].reshape(1, -1)

    # Get the nearest neighbors (i.e., similar users)
    distances, indices = model_knn.kneighbors(user_interaction_vector, n_neighbors=10)

    # Initialize a list to store the ISBNs of the recommended books
    recommended_books = []

    # Loop through the indices of the neighbors
    for idx in indices.flatten():
        # Find books that the neighbor has rated 5 stars
        high_rated_books = interaction_matrix.columns[interaction_matrix.iloc[idx] == 5].tolist()
        recommended_books.extend(high_rated_books)

    # Remove books that the user has already rated
    rated_books = interaction_matrix.columns[interaction_matrix.iloc[user_idx] > 0].tolist()
    recommended_books = list(set(recommended_books) - set(rated_books))

    # Return the recommended books
    return recommended_books

# Testing the collaborative filtering recommendation function with a user from the subset dataset
test_user_id_subset = ratings_df_subset['User-ID'].iloc[0]  # Get the first user ID for testing
collaborative_filtering_recommendations_subset = get_collaborative_filtering_recommendations(
    test_user_id_subset, interaction_matrix, model_knn
)

collaborative_filtering_recommendations_subset


['342677609X', '0805047379']

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine the 'Book-Author' and 'Publisher' columns into a single string for each book
# We'll fill any NaN values with empty strings to ensure the vectorizer works correctly
books_df['content'] = books_df['Book-Author'].fillna('') + ' ' + books_df['Publisher'].fillna('')

# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Create the TF-IDF feature matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(books_df['content'])

# Display the shape of the TF-IDF matrix to understand its size
tfidf_matrix.shape


(271360, 54504)

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

# Select a random subset of the books to avoid MemoryError
# We'll take a subset of 5000 books for the demonstration
subset_size = 5000
books_subset = books_df.sample(n=subset_size, random_state=42).reset_index(drop=True)

# Create the TF-IDF feature matrix for the subset
tfidf_matrix_subset = tfidf_vectorizer.transform(books_subset['content'])

# Compute the cosine similarity matrix from the TF-IDF matrix for the subset
cosine_sim_matrix_subset = cosine_similarity(tfidf_matrix_subset, tfidf_matrix_subset)

# Function to get book recommendations based on content similarity for the subset
def get_content_based_recommendations_subset(book_idx, books_subset, cosine_sim_matrix, top_n=10):
    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim_matrix[book_idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the top-n most similar books
    sim_scores = sim_scores[1:top_n+1]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top-n most similar books
    return books_subset.iloc[book_indices]

# Test the function with an example book index from our subset
example_index = 0  # Using the first book in the subset for testing
recommended_books_subset = get_content_based_recommendations_subset(example_index, books_subset, cosine_sim_matrix_subset)
recommended_books_subset[['ISBN', 'Book-Title', 'Book-Author', 'Publisher']]

,ISBN,Book-Title,Book-Author,Publisher
1599,0671898213,Baedeker London (Baedekers City Guides),SONS,Macmillan General Reference
1302,0394826256,Walt Disney's Snow White and the Seven Dwarfs ...,Random House Editors,Random House Childrens Books
1307,0439148790,Can You Hear a Shout in Space?: Questions and ...,Melvin Berger,Scholastic Reference
4675,0553502662,Disobedience,Jane Hamilton,Random House Audio Publishing Group
2349,0517035820,Agatha Christie: Five Classic Murder Mysteries,Agatha Christie,Random House Value Publishing
429,0679431152,Disclosure,Michael Crichton,Random House Audio Publishing Group
1294,0553473425,Bbc Presents: Middlemarch,George Eliot,Random House Audio Publishing Group
2159,0671850245,The Complete Vampire Companion,Rosemary Ellen Guiley,Macmillan General Reference
3541,0679412093,The Trail Home: Essays,John Daniel,Random House Inc
4282,0679458182,Personal History,Katharine Graham,Random House Audio Publishing Group


### Red Basado en filtros colaborativos

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Initialize label encoders
user_encoder = LabelEncoder()
book_encoder = LabelEncoder()

# Fit the encoders to the user ids and book isbns
ratings_df['User-ID'] = user_encoder.fit_transform(ratings_df['User-ID'])
ratings_df['ISBN'] = book_encoder.fit_transform(ratings_df['ISBN'])

# Check the transformed data
ratings_df_encoded_head = ratings_df.head()

# Split the data into a training set and a test set
train_data, test_data = train_test_split(ratings_df, test_size=0.2, random_state=42)

train_data.head(), test_data.head(), ratings_df_encoded_head


(         User-ID    ISBN  Book-Rating
 686565     63068  108387            0
 62456       4516   21407            0
 1122931   101831   86132            7
 636841     58113  202013            5
 878589     80185   60802            0,
          User-ID    ISBN  Book-Rating
 178554     14710   66197            0
 533905     48732  207194            8
 1091374    98946   65482            0
 1036247    93459  128975            0
 309523     28004   47683            0,
    User-ID    ISBN  Book-Rating
 0   104433   57188            0
 1   104434   29750            5
 2   104435  107392            0
 3   104436  127253            3
 4   104436  127287            6)

In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.optimizers import Adam

# Number of factors to use for embeddings
n_factors = 50

# Get the number of unique entities in users and books
n_users = ratings_df['User-ID'].nunique()
n_books = ratings_df['ISBN'].nunique()

# Book embedding model
book_input = Input(shape=(1,))
book_embedding = Embedding(n_books, n_factors, input_length=1)(book_input)
book_vec = Flatten()(book_embedding)

# User embedding model
user_input = Input(shape=(1,))
user_embedding = Embedding(n_users, n_factors, input_length=1)(user_input)
user_vec = Flatten()(user_embedding)

# Concatenate features
concat = Concatenate()([book_vec, user_vec])

# Add fully-connected layers
fc1 = Dense(128, activation='relu')(concat)
fc2 = Dense(32, activation='relu')(fc1)
output = Dense(1)(fc2)  # No activation, as this is a regression problem

# Create model and compile it
model = Model([user_input, book_input], output)
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 50)        17027800    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 50)        5264150     ['input_2[0][0]']                
                                                                                              

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Create a new dataframe for the model that includes only the necessary columns
model_data = ratings_df[['User-ID', 'ISBN', 'Book-Rating']].copy()

# Initialize the label encoders
user_le = LabelEncoder()
book_le = LabelEncoder()

# Fit the label encoders and transform the user ids and book ISBNs to encoded indices
model_data['User-ID'] = user_le.fit_transform(model_data['User-ID'])
model_data['ISBN'] = book_le.fit_transform(model_data['ISBN'])

# Split the data into training and validation sets
train_data, test_data = train_test_split(model_data, test_size=0.2, random_state=42)

# Prepare the input for the model
train_user_data = train_data['User-ID'].values
train_book_data = train_data['ISBN'].values
train_ratings = train_data['Book-Rating'].values

test_user_data = test_data['User-ID'].values
test_book_data = test_data['ISBN'].values
test_ratings = test_data['Book-Rating'].values

# Number of unique users and books
n_users = model_data['User-ID'].nunique()
n_books = model_data['ISBN'].nunique()

train_user_data.shape, train_book_data.shape, train_ratings.shape, n_users, n_books


((919824,), (919824,), (919824,), 105283, 340556)

In [15]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.optimizers import Adam

# Define the neural network model for collaborative filtering
# Number of factors to use for embeddings
n_factors = 50

# Book embedding model
book_input = Input(shape=(1,))
book_embedding = Embedding(n_books, n_factors, input_length=1)(book_input)
book_vec = Flatten()(book_embedding)

# User embedding model
user_input = Input(shape=(1,))
user_embedding = Embedding(n_users, n_factors, input_length=1)(user_input)
user_vec = Flatten()(user_embedding)

# Concatenate features
concat = Concatenate()([book_vec, user_vec])

# Add fully-connected layers
fc1 = Dense(128, activation='relu')(concat)
fc2 = Dense(32, activation='relu')(fc1)
output = Dense(1)(fc2)  # No activation, as this is a regression problem

# Create and compile the model
model = Model([user_input, book_input], output)
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

# Display the model summary
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 1, 50)        17027800    ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, 1, 50)        5264150     ['input_4[0][0]']                
                                                                                            

In [17]:
history = model.fit(
    [train_user_data, train_book_data], 
    train_ratings, 
    validation_data=([test_user_data, test_book_data], test_ratings), 
    epochs=1, 
    verbose=1
)


 1070/28745 [>.............................] - ETA: 1:53:51 - loss: 11.2315